In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time
from utils.tools import build_weighted_bethe_hessian

In [8]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
adjacency_matrix.data = adjacency_matrix.data ** 0.3
#adjacency_matrix.data /= np.max(adjacency_matrix.data)
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [9]:
#r = np.sqrt(5176.292450)
#r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
r = np.sqrt(np.mean(degrees))
Hr = build_weighted_bethe_hessian(adjacency_matrix, r)

#I = scipy.sparse.eye(n, format='csr')
#Hr = D - adjacency_matrix #+ I*np.mean(degrees)

In [13]:
# preconditioner = scipy.sparse.spdiags(1.0 / bethe_diagonal, [0], n, n, format='csr')

In [10]:
n = adjacency_matrix.shape[0]
dim = 10
tol = np.sqrt(1e-15)*n

start = time.time()
vals, vecs = eigsh(Hr, dim, which='SA', tol=tol)
print("time elapsed: %d" % (time.time() - start))

time elapsed: 38


In [7]:
### Scaled eigenvals
vals

array([-27.21834642,  -5.49479629,  -3.48974188,  -2.6891213 ,
        -2.27078449,  -2.11221216,  -1.89083085,  -1.82318882,
        -1.47392654,  -1.37102964])

In [11]:
### Not scaled eigenvals
vals

array([-581.92984447, -437.52568457, -434.62097627, -324.83389946,
       -285.70772244, -278.87931479, -252.19034481, -238.15491937,
        -90.49405409,  -52.79091689])

In [13]:
vals / np.max(adjacency_matrix.data)

array([-4.15374624, -3.12300647, -3.102273  , -2.31862587, -2.03934785,
       -1.99060749, -1.80010478, -1.69992158, -0.64593583, -0.37681531])

In [13]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_scaled2_pow=0.30_dim=100"
np.save(output_path + ".vecs", vecs)
np.save(output_path + ".vals", vals - np.mean(degrees))
np.save(output_path + ".degrees", degrees)

In [14]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_scaled3_pow=0.30_dim=100"
np.save(output_path + ".vecs", vecs2)
np.save(output_path + ".vals", vals2 - np.mean(degrees))
np.save(output_path + ".degrees", degrees)

In [11]:
output_path = "../data/wiki/win=1_bethe_hessian_small_rhoB_est_pow=0.00_dim=100"
vecs2 = np.load(output_path + ".vecs.npy")
vals2 = np.load(output_path + ".vals.npy")

In [14]:
all_vals = np.concatenate([vals, vals2], axis=0)
all_vecs = np.concatenate([vecs, vecs2], axis=1)

In [19]:
top_vals_inds = np.argsort(all_vals)[:100]
output_path = "../data/wiki/win=1_bethe_hessian_combo_rhoB_est_pow=0.00_dim=100"
np.save(output_path + ".vecs", all_vecs[:, top_vals_inds])
np.save(output_path + ".vals", all_vals[top_vals_inds])
np.save(output_path + ".degrees", degrees)